The dataset we will be working with compiles detailed information on international football matches from 1872 to 2024, covering over 47,000 matches between national teams. It includes details such as participating teams, dates, locations, tournaments, and results, allowing for an analysis of the evolution of international football over more than a century.

The dataset is divided into four CSV files:

results.csv: Record of matches including date, teams, score, and location.
shootouts.csv: Information on penalty shootouts in decisive matches.
former_names.csv: History of country names that have changed over time.
scorers.csv: List of players who have scored goals in each match.
This database provides a wide range of information, enabling the study of trends in national team performance, changes in game dynamics, and the influence of tournaments over the years.

The data comes from historical records and has been organized on Kaggle to facilitate analysis.

Kaggle link:
https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017/data

In [3]:
import pandas as pd
import os

Load csvs into dataframes dictionary

In [4]:
file_paths = {
    "former_names": "../csvs/former_names.csv",
    "goalscorers": "../csvs/goalscorers.csv",
    "results": "../csvs/results.csv",
    "shootouts": "../csvs/shootouts.csv"
}

dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

## Aux Functions

In [5]:
def check_former_names(dataframes, dataset_name):

    df = dataframes[dataset_name]
    former_names_list = set(dataframes["former_names"]["former"])

    # Identify columns where former names might appear
    columns_to_check = [col for col in df.columns if col in ["home_team", "away_team", "country"]]
    
    # Store the found names in each column along with their counts
    former_names_found = {}

    for col in columns_to_check:
        found_names = df[col][df[col].isin(former_names_list)].value_counts()
        if not found_names.empty:
            former_names_found[col] = found_names.to_dict()

    if not former_names_found:
        print(f"No former names found in {dataset_name}.")
        return None

    return former_names_found


def replace_former_names(dataframes, dataframe_name):

    # Exclude Soviet Union and CIS from mapping
    excluded_countries = {"Soviet Union", "CIS"}
    mapping = {row["former"]: row["current"] for _, row in dataframes["former_names"].iterrows() if row["former"] not in excluded_countries}

    # Replace in home_team, away_team, and country columns if they exist
    df = dataframes[dataframe_name]
    for col in ["home_team", "away_team", "country"]:
        if col in df.columns:
            df[col] = df[col].replace(mapping)

    return df

## Csv Cleaning

#### Former_names.csv

I will delete this CSV later because I believe it does not contribute much to the study objective of my dataset. Additionally, it may cause some confusion since there are countries like the USSR or the CIS that were later renamed as Russia. This is somewhat confusing as they were also formed by other countries.

However, to avoid losing the information completely and to provide greater consistency to the data, I will map the former names to the current names in the other CSVs, excluding cases categorized as 'confusing'.

In [6]:
# List of datasets to check
datasets_to_check = ["results", "goalscorers", "shootouts"]

# Iterate over each dataset and apply the function
for dataset in datasets_to_check:
    former_names_found = check_former_names(dataframes, dataset)
    print(f"\nFormer names found in {dataset}.csv:")
    if former_names_found:
        for column, names in former_names_found.items():
            print(f"  - In column '{column}':")
            for name, count in names.items():
                print(f"    {name}: {count} occurrences")
    else:
        print("  No former names found.")


Former names found in results.csv:
  - In column 'home_team':
    Burma: 4 occurrences
    Malaya: 1 occurrences
  - In column 'away_team':
    Burma: 3 occurrences
    Malaya: 1 occurrences
  - In column 'country':
    Soviet Union: 167 occurrences
    Swaziland: 110 occurrences
    Malaya: 95 occurrences
    Dutch Guyana: 81 occurrences
    Zaïre: 59 occurrences
    Ireland: 53 occurrences
    Burma: 36 occurrences
    British Guiana: 29 occurrences
    Western Samoa: 27 occurrences
    Tanganyika: 24 occurrences
    Upper Volta: 22 occurrences
    Southern Rhodesia: 17 occurrences
    Éire: 17 occurrences
    Dahomey: 16 occurrences
    Congo-Kinshasa: 16 occurrences
    Northern Rhodesia: 12 occurrences
    Irish Free State: 9 occurrences
    Ceylon: 8 occurrences
    Gold Coast: 6 occurrences
    Nyasaland: 4 occurrences
    French Somaliland: 4 occurrences
    New Hebrides: 3 occurrences
    Mandatory Palestine: 3 occurrences
    Bohemia: 3 occurrences
    Belgian Congo: 3 occur

We see that former names only appear in results.csv, so we will apply the mapping only to this CSV

In [8]:
dataframes["results"] = replace_former_names(dataframes, "results") 

Check if mapping was succesfull

In [9]:
# List of datasets to check
datasets_to_check = ["results", "goalscorers", "shootouts"]

# Iterate over each dataset and apply the function
for dataset in datasets_to_check:
    former_names_found = check_former_names(dataframes, dataset)
    print(f"\nFormer names found in {dataset}.csv:")
    if former_names_found:
        for column, names in former_names_found.items():
            print(f"  - In column '{column}':")
            for name, count in names.items():
                print(f"    {name}: {count} occurrences")
    else:
        print("  No former names found.")


Former names found in results.csv:
  - In column 'country':
    Soviet Union: 167 occurrences
No former names found in goalscorers.

Former names found in goalscorers.csv:
  No former names found.
No former names found in shootouts.

Former names found in shootouts.csv:
  No former names found.


#### Results.csv

In [10]:
dataframes["results"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47980 entries, 0 to 47979
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        47980 non-null  object
 1   home_team   47980 non-null  object
 2   away_team   47980 non-null  object
 3   home_score  47980 non-null  int64 
 4   away_score  47980 non-null  int64 
 5   tournament  47980 non-null  object
 6   city        47980 non-null  object
 7   country     47980 non-null  object
 8   neutral     47980 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 3.0+ MB


We convert the date column so that it is not a string but a datetime format, making searches easier. (We take this opportunity to apply it to all CSV files.)

In [7]:
dataframes["results"]["date"] = pd.to_datetime(dataframes["results"]["date"])
dataframes['goalscorers']['date'] = pd.to_datetime(dataframes['goalscorers']['date'])
dataframes['shootouts']['date'] = pd.to_datetime(dataframes['shootouts']['date'])

In [11]:
dataframes['results'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47980 entries, 0 to 47979
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        47980 non-null  datetime64[ns]
 1   home_team   47980 non-null  object        
 2   away_team   47980 non-null  object        
 3   home_score  47980 non-null  int64         
 4   away_score  47980 non-null  int64         
 5   tournament  47980 non-null  object        
 6   city        47980 non-null  object        
 7   country     47980 non-null  object        
 8   neutral     47980 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(5)
memory usage: 3.0+ MB


#### Shootouts.csv

In [8]:
dataframes['shootouts'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           645 non-null    datetime64[ns]
 1   home_team      645 non-null    object        
 2   away_team      645 non-null    object        
 3   winner         645 non-null    object        
 4   first_shooter  230 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 25.3+ KB


In [9]:
dataframes['shootouts'].isnull().sum()

date               0
home_team          0
away_team          0
winner             0
first_shooter    415
dtype: int64

We keep the first_shooter column despite the high presence of null values, as I believe it can provide relevant insights for the analysis of penalty shootouts. I think studying the influence of shooting order on the probability of victory could reveal interesting patterns, such as a potential advantage for the team that shoots first. Additionally, this data will allow me to analyze in which time periods or competitions there are more null values, which could offer clues about the evolution of data recording in tournaments.

#### Goalscorers.csv

In [10]:
dataframes['goalscorers'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44362 entries, 0 to 44361
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       44362 non-null  datetime64[ns]
 1   home_team  44362 non-null  object        
 2   away_team  44362 non-null  object        
 3   team       44362 non-null  object        
 4   scorer     44313 non-null  object        
 5   minute     44103 non-null  float64       
 6   own_goal   44362 non-null  bool          
 7   penalty    44362 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float64(1), object(4)
memory usage: 2.1+ MB


In [11]:
dataframes['goalscorers'].isnull().sum()

date           0
home_team      0
away_team      0
team           0
scorer        49
minute       259
own_goal       0
penalty        0
dtype: int64

We get rid of nulls as they are difficult to fill and not many compared to the number of rows we have.

In [12]:
dataframes['goalscorers'] = dataframes['goalscorers'].dropna(subset=['scorer', 'minute'])

# Verificar que los valores nulos fueron eliminados
print(dataframes['goalscorers'].isnull().sum())
print(f"Rows remaining: {len(dataframes['goalscorers'])}")

date         0
home_team    0
away_team    0
team         0
scorer       0
minute       0
own_goal     0
penalty      0
dtype: int64
Rows remaining: 44102


In [13]:
# Directorio donde quieres guardar los archivos limpios
save_directory = "../csvs/"

# Lista de dataframes que queremos guardar
clean_dataframes = {
    'results': 'results_clean.csv',
    'shootouts': 'shootouts_clean.csv',
    'goalscorers': 'goalscorers_clean.csv'
}

# Guardar cada dataframe limpio como CSV en el directorio correcto
for key, filename in clean_dataframes.items():
    if key in dataframes:  # Verificar que el dataframe existe en el diccionario
        filepath = os.path.join(save_directory, filename)
        dataframes[key].to_csv(filepath, index=False)